## Imports & Mounting Google Drive


We start by mounting Google Drive to access the necessary datasets and files directly. This is essential for working with the spectrogram images, metadata, and any additional resources required for training the CNN model.



In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import h5py  # for saving to HDF5 format


from google.colab import drive
drive.mount('/content/drive')  # Mount Google Drive for accessing files


# Path Definitions
We define paths to the directories containing spectrogram images, additional datasets, and the labels CSV file. These paths are crucial for loading data and associating labels with corresponding spectrograms.

In [ ]:
# Define paths
# Assuming all spectrogram images are stored in a directory called 'spectrogram_dir'
spectrogram_dir = '/content/drive/MyDrive/COSC_5470/spectrograms' #'/Users/sophiacastor/Dev/School/Y3_S24/COSC_5470/spectrograms'
art_dataset_dir = '/content/drive/MyDrive/COSC_5470/AI_Abstract_Surrealism_DataSet' #'/Users/sophiacastor/Dev/School/Y3_S24/COSC_5470/AI_LD_surrealism'
label_csv_path = '/content/drive/MyDrive/COSC_5470/song_metadata.csv' #'/Users/sophiacastor/Dev/School/Y3_S24/COSC_5470/song_metadata.csv'

# Loading Labels
We define a function to read and process the labels from a CSV file. This function cleans up the data, mapping song titles to their valence and genre, and also creates a mapping for energy labels. This is important for preparing labels that will be used in training.

In [ ]:
def load_labels(label_file):
    labels_df = pd.read_csv(label_file)
    labels_df['Title'] = labels_df['Title'].apply(lambda x: x.strip())  # Clean up whitespace
    file_to_valence_genre = labels_df.set_index('Title')[['Valence', 'Genre']].to_dict('index')
    file_to_energy = labels_df.set_index('Title')['Energy'].to_dict()
    return labels_df, file_to_valence_genre, file_to_energy

labels_df, file_to_valence_genre, file_to_energy = load_labels(label_csv_path)
print (labels_df)
print (file_to_valence_genre)
print (file_to_energy)

# Matching Labels to Spectrogram Files
We implement a function to match labels with corresponding spectrogram files. This function iterates over the song titles in the DataFrame and checks for matching files in the spectrogram directory, recording any unmatched titles for later review.

In [ ]:
import pandas as pd
import os

def match_labels_to_files(df, spectrogram_dir):
    """
    Matches each song name in the DataFrame with its corresponding spectrogram in the directory.

    :param df: DataFrame containing song titles.
    :param spectrogram_dir: Directory containing spectrogram files.
    :return: Tuple of matched files and unmatched files.
    """
    matched_files = {}
    unmatched_files = []

    # Create a set of spectrogram filenames (without the .png extension) for easy lookup
    spectrogram_files = set(os.path.splitext(file)[0].strip().lower() for file in os.listdir(spectrogram_dir))

    for idx, row in df.iterrows():
        title = row['Title'].strip().lower()

        if title in spectrogram_files:
            matched_files[title] = os.path.join(spectrogram_dir, f"{title}.png")
        else:
            unmatched_files.append(title)

    print(f'Matched files: {len(matched_files)}')
    print(f'Unmatched files: {len(unmatched_files)}')

    return matched_files, unmatched_files

# Usage Example:
label_csv_path = '/content/drive/MyDrive/COSC_5470/song_metadata.csv'
spectrogram_dir = '/content/drive/MyDrive/COSC_5470/spectrograms' #'/Users/sophiacastor/Dev/School/Y3_S24/COSC_5470/spectrograms'

# Load Labels
labels_df = pd.read_csv(label_csv_path)

# Match Labels to Files
matched_files, unmatched_files = match_labels_to_files(labels_df, spectrogram_dir)


# Loading and Preprocessing Spectrograms
This function loads spectrogram images and preprocesses them into arrays, normalizing the pixel values and mapping them to labels. The resulting arrays are then split into training and testing sets for later model training.

In [ ]:
# Function to match spectrograms with labels and load them at their original resolution
def load_and_preprocess_spectrograms(spectrogram_dir, labels_df):
    X = []  # Spectrogram data
    y_valence = []  # Valence labels
    y_energy = []  # Energy labels

    for index, row in labels_df.iterrows():
        file_path = os.path.join(spectrogram_dir, row['Title'] + '.png')
        if os.path.exists(file_path):
            image = load_img(file_path, color_mode='rgb')  # Load at original resolution
            image_array = img_to_array(image) / 255.0  # Normalize the image
            X.append(image_array)
            y_valence.append(row['Valence'])
            y_energy.append(row['Energy'])

    return np.array(X), np.array(y_valence), np.array(y_energy)

# Load and preprocess the data
X, y_valence, y_energy = load_and_preprocess_spectrograms(spectrogram_dir, labels_df)

# Split the dataset into training and testing
X_train, X_test, y_valence_train, y_valence_test, y_energy_train, y_energy_test = train_test_split(
    X, y_valence, y_energy, test_size=0.2, random_state=42
)

# Print a few preprocessed spectrograms to verify
plt.figure(figsize=(10, 5))
for i in range(min(5, len(X_train))):
    plt.subplot(1, 5, i+1)
    plt.imshow(X_train[i])
    plt.axis('off')
plt.show()

In [ ]:
# Load and preprocess the data
X, y_valence, y_energy = load_and_preprocess_spectrograms(spectrogram_dir, labels_df)

# Split the dataset into training and testing
X_train, X_test, y_valence_train, y_valence_test, y_energy_train, y_energy_test = train_test_split(
    X, y_valence, y_energy, test_size=0.2, random_state=42
)

In [ ]:
# Print out the shapes and some statistics
print("Spectrogram data shape:", X.shape)
print("Valence label shape:", y_valence.shape)
print("Energy label shape:", y_energy.shape)
print("Spectrogram data - min:", np.min(X), "max:", np.max(X))
print("Valence labels - min:", np.min(y_valence), "max:", np.max(y_valence))
print("Energy labels - min:", np.min(y_energy), "max:", np.max(y_energy))

# Print out the shapes of the training and testing sets
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)
print("Training valence shape:", y_valence_train.shape)
print("Testing valence shape:", y_valence_test.shape)
print("Training energy shape:", y_energy_train.shape)
print("Testing energy shape:", y_energy_test.shape)


# CNN Architecture
We define a Convolutional Neural Network (CNN) model for classifying spectrograms into valence and energy labels. The model consists of several convolutional layers, pooling layers, and dropout layers, ending with a dense layer for outputting valence and energy labels.

In [ ]:
# Define the CNN architecture
def build_cnn_classifier(input_shape):
    input_layer = Input(shape=input_shape)
    x = Conv2D(32, (5, 5), activation='relu', padding='same', kernel_regularizer=l2(0.01))(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.3)(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.3)(x)

    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Change to a single output layer with 2 units for valence and energy
    outputs = Dense(2, activation='linear')(x)

    model = Model(inputs=input_layer, outputs=outputs)
    model.compile(optimizer=RMSprop(learning_rate=0.0001, momentum=0.9),
              loss='mean_squared_error',  # Use one loss for the entire output
              metrics=['mean_squared_error'])
    return model

# Initial

# Data Augmentation
We set up an ImageDataGenerator to augment the training data. This includes applying rotations, shifts, shearing, and other transformations to the spectrograms to make the model more robust.

In [ ]:
# Set up data augmentation for the training set
datagen = ImageDataGenerator(
    rotation_range=10,  # Increase rotation range
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2),  # New augmentation for brightness
    fill_mode='nearest'
)
# datagen = ImageDataGenerator()



In [ ]:
# Initialize the model
input_shape = X_train[0].shape
model = build_cnn_classifier((800, 1200, 3))  # Use the correct input shape

# Training Parameters and Callbacks
We define training parameters such as batch size and epochs, along with callbacks like ModelCheckpoint, EarlyStopping, and LearningRateScheduler. These callbacks manage the training process, stopping it early if needed and saving the best-performing model.

In [ ]:
# Set up the training parameters
batch_size = 16
epochs = 50

# Define the path to save the best model
checkpoint_path = 'best_model.keras'

# Combine the valence and energy labels for training into a single array
y_combined_train = np.hstack((y_valence_train.reshape(-1, 1), y_energy_train.reshape(-1, 1)))

# Combine the valence and energy labels for testing into a single array
y_combined_test = np.hstack((y_valence_test.reshape(-1, 1), y_energy_test.reshape(-1, 1)))

# Create ModelCheckpoint callback to save the best model during training
checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Create EarlyStopping callback to halt training when validation loss stops improving
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True
)

# Function to update the learning rate
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

# Create LearningRateScheduler callback
lr_scheduler = LearningRateScheduler(scheduler, verbose=1)






# Model Training
We train the model using the defined architecture, dataset, and callbacks, monitoring both training and validation losses.

In [ ]:
# Train the model
history = model.fit(
    datagen.flow(X_train, y_combined_train, batch_size=batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    validation_data=(X_test, y_combined_test),
    callbacks=[checkpoint, early_stopping, lr_scheduler]
)



# Saving the Model
We save the trained model in both Keras and HDF5 formats for future use.

In [ ]:
# Save the final model
model.save('/content/drive/MyDrive/COSC_5470/trained_cnn_model.keras')
model.save('/content/drive/MyDrive/COSC_5470/trained_cnn_model.h5')


# Loss Visualization
We plot the training and validation losses over epochs to visualize the model's learning process.

In [ ]:

# Plotting training and validation loss
import matplotlib.pyplot as plt

# Plotting training and validation loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()




---

